---
# Setup

In [5]:
import polars as pl
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from time import sleep
import datetime
import sqlite3
import requests
import json
from JulianFinal import Cosine_Similarity
from JulianFinal2 import Cosine_Similarity2



ImportError: cannot import name 'Cosine_Similarity2' from 'JulianFinal2' (c:\Users\julia\OneDrive\Desktop\netflix_recommendation\Source\JulianFinal2.py)

In [18]:
cosine_similarity = Cosine_Similarity()


c:\Users\julia\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\julia\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [3]:
# small size for testing (100.000 rows)
db_dev_path = 'netflix_dev.db'
db_dev_conn = 'sqlite://' + db_dev_path

# full size for production (~100.000.000 rows)
db_prod_path = 'netflix.db'
db_prod_conn = 'sqlite://' + db_prod_path

netflix_data = pl.read_database("SELECT * FROM netflix_data", db_prod_conn)
movie_titles = pl.read_database("SELECT * FROM movie_titles", db_dev_conn)

In [5]:
netflix_data

film,user,rating,date
i64,i64,i64,str
1,1488844,3,"""2005-09-06 """
1,822109,5,"""2005-05-13 """
1,885013,4,"""2005-10-19 """
1,30878,4,"""2005-12-26 """
1,823519,3,"""2004-05-03 """
1,893988,3,"""2005-11-17 """
1,124105,4,"""2004-08-05 """
1,1248029,3,"""2004-04-22 """
1,1842128,4,"""2004-05-09 """


---
# Functions

Implement your recommendation engine logic in `predict_rating()`. Then, call
`full_evaluation()` to evaluate your engine.

| Function | Description |
| --- | --- |
| `get_user_ratings(user_id)` | Returns a DataFrame of all ratings for a given user. |
| `ratings_to_vector(ratings, placeholder=-1)` | Transforms a DataFrame of ratings into a vector. |
| `mask_ratings(ratings, split)` | Mask ratings in a user's rating DataFrame. Works like a training-/test-set split |
| `predict_rating(user_id, movie_id)` | Predict how a user would rate a movie. **IMPLEMENT YOUR ENGINE HERE**|
| `rate_all(user_id)` | Predict ratings of all movies for a given user |
| `get_top_rated(ratings, split)` | Get the top rated movies for a user. |
| `evaluate(masked_ratings, top_ratings)` | Evaluate the accuracy of the predictions. |
| `full_evaluation(user_id, split_mask, split_top)` | Like evaluate() but contains all the steps. |

In [4]:
# get all ratings for a specific user
def get_user_ratings(user_id):
    return netflix_data.filter(pl.col("user") == user_id)

In [5]:
# get all ratings for a specific user with rating > 3
def get_highly_rated_user_ratings(user_id):
    return netflix_data.filter((pl.col("user") == user_id) & (pl.col("rating") >= 4))


In [6]:
# ratings to vector
def ratings_to_vector(ratings, placeholder=-1):
    """
    Convert a DataFrame of ratings to a vector of ratings

    Args:
        ratings (DataFrame): DataFrame of ratings
        placeholder (int, optional): Placeholder value for missing ratings. Defaults to -1.

    Returns:
        list[float]: List of ratings
    """

    ratings_list = [placeholder] * len(movie_titles)

    # get the ratings as a numpy array
    for current_rating in ratings.rows():

        film = (current_rating[0]) - 1   # film id starts at 1, but list index starts at 0, so subtract 1
        rating = current_rating[2]
        ratings_list[film] = rating

    return ratings_list


In [7]:
# split ratings into masked and unmasked ratings based on a date
# all ratings before the date are unmasked, all ratings after the date are masked
def mask_ratings(ratings, split_date):
    """Mask ratings in a user's rating DataFrame

    Args:
        ratings (pl.DataFrame): A User's ratings
        split_date (datetime.datetime): Date to split ratings on

    Returns:
        [pl.Dataframe, pl.Dataframe]: Masked ratings, unmasked ratings
    """

    unmasked_ratings = []
    masked_ratings = []

    for rating in ratings.rows():

        # get the ratings date as datetime
        rating_date = datetime.datetime.strptime(rating[3][:-1], "%Y-%m-%d")

        # if the rating is before the split date, mask it
        if rating_date > split_date:
            unmasked_ratings.append(rating)
        else:
            masked_ratings.append(rating)

    # convert the lists to DataFrames
    unmasked_ratings = pl.DataFrame(unmasked_ratings, schema=ratings.columns)
    masked_ratings = pl.DataFrame(masked_ratings, schema=ratings.columns)

    #keep only masked_ratings with a score of 4 or higher
    masked_ratings = masked_ratings.filter(pl.col("rating") >= 4)

    return masked_ratings, unmasked_ratings

In [8]:
# sort sums by score
def sort_sums(sums):

    # convert the sums to a list of tuples
    sums_list = []
    for i in range(len(sums)):
        sums_list.append((i, sums[i]))

    # sort the list
    sums_list.sort(key=lambda x: x[1], reverse=True)

    return sums_list

In [9]:
# predict how a user would rate a movie
import random


def predict_rating(user_id, movie_id):
    """Predict how a user would rate a movie
    
    Args:
        user_id (int): User ID
        movie_id (int): Movie ID

    Returns:
        float: Predicted rating
    """

    ### PUT YOUR EVALUATION CODE HERE ###

    return random.uniform(1, 5)

In [19]:
# get cosine similarities for all movies
def get_cosine_similarities(movie_id):
    """
    Get cosine similarities for all movies
    
    Args:
        movie_id (int): Movie ID
        
    Returns:
        pl.Series: Series of cosine similarities
    """

    scores = []

    # go through all movies
    for movie in movie_titles["film"]:
        
        # calculate cosine similarity
        score = cosine_similarity.get_movie_cos_scores(movie)
        
        scores.append(score)

        if movie == movie_id:
            score = 0

    scores = pl.Series("scores", scores)

    return scores
    

In [11]:
def sum_cosine_similarities(unmasked_ratings):

    # get the user_id from unmasked_ratings
    user_id = [unmasked_ratings["user"][0]] * len(movie_titles["film"])

    sum_scores = [0] * len(movie_titles["film"])
    sum_scores = pl.Series("sum_scores", sum_scores)

    for rating in unmasked_ratings.rows():

        if rating[2] >= 3:

            scores = get_cosine_similarities(rating)
            
            sum_scores = sum_scores + scores

    # join the scores with the movie_ids from movie_titles
    sum_scores = pl.DataFrame({"user": user_id, "film": movie_titles["film"], "rating": sum_scores})
            
    return sum_scores

In [12]:
# rate all movies for a given user
def rate_all(user_id):
    """Returns a dataframe with all ratings for a given user

    Args:
        user_id (int): User ID

    Returns:
        pl.DataFrame: Ratings for a given user
    """


    # get all movies
    movies = movie_titles["film"]
    ratings = []

    for movie in movies:
        # predict rating
        rating = predict_rating(user_id, movie)

        # add rating to dataframe
        ratings.append([user_id, movie, rating])

    ratings = pl.DataFrame(ratings, schema=["user", "film", "rating"])

    return ratings

In [13]:
# get the top n ratings for a given user
def get_top_rated(ratings, n):
    """Get the top n of ratings for a given user

    Args:
        ratings (pl.DataFrame): Ratings
        n (int): Number of ratings to get

    Returns:
        pl.DataFrame: Top n ratings
    """

    # sort ratings by rating
    ratings = ratings.sort(by="rating", descending=True)

    # take the top n ratings
    top_ratings = ratings.head(n)

    return top_ratings

In [14]:
# evaluate how well a model performs
# for this, we compare how well the model predicts the top m% of ratings
# we calculate the intersection between the masked ratings and the top m% of ratings
def evaluate(masked_ratings, top_ratings):
    """Evaluate how well a model performs

    Args:
        masked_ratings (pl.DataFrame): Masked ratings
        top_ratings (pl.DataFrame): Top m% of ratings

    Returns:
        float: Percentage of ratings that were correctly predicted
    """

    # get intersection between masked ratings and top ratings
    intersection = masked_ratings.join(top_ratings, on=["user", "film"], how="inner")

    # calculate percentage of ratings that were correctly predicted
    percentage = len(intersection) / len(masked_ratings)

    return percentage

In [15]:
# all steps combined
# TODO: update the parameters
def full_evaluation(user_id, split_mask, split_top):
    """Evaluate how well a model performs (for a given user)

    Args:
        user_id (int): User ID
        split_mask (float): Percentage of ratings to mask
        split_top (float): Percentage of top ratings to compare with the masked ratings

    Returns:
        float: Percentage of ratings that were correctly predicted
    """

    masked_ratings, unmasked_ratings = mask_ratings(get_user_ratings(user_id), split_mask)

    all_ratings = rate_all(user_id)
    top_ratings = get_top_rated(all_ratings, split_top)

    evaluation =  evaluate(masked_ratings, top_ratings)

    return evaluation

In [16]:
def full_evaluate_cosine_similarity(user_id, split_date, top_n):
    """Evaluate how well a model performs (for a given user)

    Args:
        user_id (int): User ID
        split_date (datetime.datetime): Date to split ratings on
        top_n (int): Number of top ratings to compare with the masked ratings

    Returns:
        float: Percentage of ratings that were correctly predicted
    """

    # get all ratings for user_id
    ratings = get_highly_rated_user_ratings(user_id)

    # split ratings into masked and unmasked ratings
    unmasked_ratings, masked_ratings = mask_ratings(ratings, split_date)

    # get sum of cosine similarities for unmasked ratings
    sums = sum_cosine_similarities(unmasked_ratings)

    # get top n ratings
    top_ratings = get_top_rated(sums, top_n)

    # evaluate
    evaluation = evaluate(masked_ratings, top_ratings)

    return evaluation

---
# Testing

In [18]:
# find the 5 users with the most ratings
user_rating_counts = netflix_data.groupby("user").count().sort(by="count", descending=True)
user_rating_counts.head(5)

user,count
i64,u32
305344,17653
387418,17436
2439493,16565
1664010,15813
2118461,14831


In [ ]:
# get all ratings for a user
get_user_ratings(10)

film,user,rating,date
i64,i64,i64,str
13580,7,4,"""2005-05-23 """


In [ ]:
# transform the ratings into a vector
# here, all missing ratings get the placeholder value 0
ratings_to_vector(get_user_ratings(7), 0)

In [214]:
# evaluate the model for a user
full_evaluation(2118461, 0.5, 0.1)

0.14285714285714285

In [20]:
split_date = datetime.datetime(2005, 9, 1)
full_evaluate_cosine_similarity(10, split_date, 500)

In [ ]:
split_date = datetime.datetime(2005, 9, 1)
full_evaluate_cosine_similarity(305344, split_date, 500)